In [ ]:
import os
import cv2
import optuna
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
%cd ..

In [ ]:
from src.transform import get_extrinsic_matrix
from src.camera import load_image, undistort_image
from src.calibration import find_x_coordinate, get_calibration_points, get_calibration_error
from src.aruco import get_aruco_marker_corners, get_aruco_marker_inner_corners, draw_aruco_markers
from src.lidar import load_lidar_data, process_lidar_data, filter_lidar_data, get_lidar_terminal_points

In [ ]:
def get_img_mask(points, img_dim):
    h, w = img_dim
    w_mask = (0 <= points[:, 0]) & (points[:, 0] < w)
    h_mask = (0 <= points[:, 1]) & (points[:, 1] < h)
    mask = w_mask & h_mask
    return mask

In [ ]:
def save_parameters(path, **kwargs):
    np.savez(path, **kwargs)
    print("Parameters:", list(kwargs.keys()))
    print("Parameters saved to:", path)

In [ ]:
def calibrate(roll_deg, pitch_deg, yaw_deg, x0, y0, z0, scale_x, scale_y, trans_x, trans_y):    
    global images, lidar_data, cam_matrix, dist_coeffs
    
    scale_mat = np.array([
        [scale_x, 0],
        [0, scale_y],
    ])
    trans_vec = np.array([trans_x, trans_y])
    extrinsic_mat = get_extrinsic_matrix(roll_deg, pitch_deg, yaw_deg, x0=x0, y0=y0, z0=z0)
    
    errors = []
    for img, lidar in zip(images, lidar_data):
        img = img.copy()
        img = undistort_image(img, cam_matrix, dist_coeffs)
        marker_corners = get_aruco_marker_corners(img)
        
        lidar_pixels = process_lidar_data(lidar, extrinsic_mat, cam_matrix, dist_coeffs, scale_mat, trans_vec)
        lidar_terminal_points = get_lidar_terminal_points(lidar_pixels)
        cal_points = get_calibration_points(marker_corners, lidar_terminal_points)
        cal_error = get_calibration_error(cal_points, lidar_terminal_points)
        errors.append(cal_error)
    return np.mean(errors)

In [ ]:
def obj_function(trial):
    # Optimized parameters
    roll_deg = trial.suggest_int("roll_deg", 70, 100)
    pitch_deg = trial.suggest_int("pitch_deg", 260, 280)
    yaw_deg = trial.suggest_int("yaw_deg", 170, 200)
    x0 = trial.suggest_int("x0", -15, 50)
    y0 = trial.suggest_int("y0", -15, 50)
    z0 = trial.suggest_int("z0", -15, 50)
    scale_x = trial.suggest_float("scale_x", 1.0, 3.0)
    trans_x = trial.suggest_int("trans_x", -300, 0)
    # trans_y = trial.suggest_int("trans_y", -100, 0)
    
    # Constant parameters
    # x0, y0, z0 = 10, 30, 40
    scale_y = 1.0
    trans_y = -50
    return calibrate(roll_deg, pitch_deg, yaw_deg, x0, y0, z0, scale_x, scale_y, trans_x, trans_y)

# Load data

In [ ]:
path2data = Path("data/calibration_data/")

num_images = len(list(path2data.glob("*.jpg")))
num_lidar_data = len(list(path2data.glob("*.npy")))
assert num_images == num_lidar_data

num_data = num_images

assert num_data > 0, "No data found!"

In [ ]:
images = []
lidar_data = []
for i in tqdm(range(num_data), desc="Loading Data"):
    path2image = path2data / f"data_{i+1}.jpg"
    path2lidar = path2data / f"data_{i+1}.npy"
    img = load_image(path2image)
    lidar_points = load_lidar_data(path2lidar)
    lidar_points = filter_lidar_data(lidar_points)
    images.append(img)
    lidar_data.append(lidar_points)

# Load Camera Intrinsic Parameter

In [ ]:
path2intrinsic_params = Path("data/parameters/camera_intrinsic_params.npz")
int_params = np.load(path2intrinsic_params)
cam_matrix = int_params["camera_matrix"]
cam_matrix

In [ ]:
dist_coeffs = int_params["dist_coeffs"]
dist_coeffs

# Search parameters

In [ ]:
study = optuna.create_study(
    study_name="lidar_param_study",
    storage="sqlite:///lidar_param_1.db",
    load_if_exists=True,
    direction="minimize",
    sampler=optuna.samplers.TPESampler(n_startup_trials=100, n_ei_candidates=200),
)

In [ ]:
for _ in range(8):
    study.optimize(obj_function, n_trials=5000, show_progress_bar=True)
    
    trial = study.best_trial
    print("Best Score: ", trial.value)
    print("Best Params: ")
    for key, value in trial.params.items():
        print("  {}: {}".format(key, value))